In [1]:
# import librries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
# import coin datas
import yfinance as yf
xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     start = "2022-09-07",
                     end = "2023-09-09" , 
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [3]:
df_xmr.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-09-04,140.842804,143.770203,140.776291,141.170868,141.170868,63835398
2023-09-05,141.176682,141.299179,137.673279,137.781372,137.781372,54468002
2023-09-06,137.780243,142.644989,137.635544,142.113235,142.113235,67734044
2023-09-07,142.107117,143.275955,141.168930,143.250748,143.250748,62422431
2023-09-08,143.257370,144.124344,140.928955,143.886154,143.886154,46927009


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score , r2_score , recall_score , precision_score

In [7]:
# find best number of neighbors and number of days for accuracy
n_days = 15
n_neigh = 8
for h in range(1,n_days) :
    f_sc = 0 
    r_sc = 0
    p_sc = 0
    nn = 0
    for t in range(2,n_neigh) :
        kll = []
        y_change = []
        for i in range(h,len(df_xmr)) : 
            g =[ df_xmr.iloc[j]["Close"] for j in range(i-h,i) ]
            kll.append(g)
        for i in range(1,len(df_xmr)) : 
            y_change.append(df_xmr.iloc[i]["Close"] > df_xmr.iloc[i-1]["Close"])
        knn_10 = KNeighborsClassifier(n_neighbors=t)
        knn_10.fit(kll,y_change[h-1:])
        y_10 = knn_10.predict(kll)
        ff_sc = f1_score(y_true=y_change[h-1:] , y_pred = y_10)
        rr_sc = recall_score(y_true=y_change[h-1:] , y_pred = y_10)
        pp_sc = precision_score(y_true=y_change[h-1:] , y_pred = y_10)
        nn = t if ff_sc > f_sc else nn
        f_sc = ff_sc if ff_sc > f_sc else f_sc
        r_sc = rr_sc if rr_sc > r_sc else r_sc
        p_sc = pp_sc if pp_sc > p_sc else p_sc
    print("n_days:" , h, "     n_neighbors:",  nn  , ":" )
    print(f_sc)
    print(r_sc)
    print(p_sc)

n_days: 1      n_neighbors: 3 :
0.7813267813267815
0.7871287128712872
1.0
n_days: 2      n_neighbors: 3 :
0.8179669030732861
0.8606965174129353
1.0
n_days: 3      n_neighbors: 3 :
0.7549019607843137
0.77
1.0
n_days: 4      n_neighbors: 3 :
0.8066825775656324
0.845
1.0
n_days: 5      n_neighbors: 3 :
0.775
0.7788944723618091
1.0
n_days: 6      n_neighbors: 3 :
0.7405541561712846
0.7525252525252525
1.0
n_days: 7      n_neighbors: 3 :
0.7692307692307692
0.7828282828282829
1.0
n_days: 8      n_neighbors: 3 :
0.7761194029850746
0.7969543147208121
1.0
n_days: 9      n_neighbors: 3 :
0.7673267326732673
0.7918781725888325
1.0
n_days: 10      n_neighbors: 3 :
0.7642679900744416
0.8061224489795918
1.0
n_days: 11      n_neighbors: 3 :
0.7794117647058825
0.8112244897959183
1.0
n_days: 12      n_neighbors: 3 :
0.7804878048780487
0.8163265306122449
1.0
n_days: 13      n_neighbors: 3 :
0.7665847665847666
0.8112244897959183
1.0
n_days: 14      n_neighbors: 3 :
0.7531172069825437
0.7743589743589744
1.0

In [9]:
# create model with best params
best_n_days = 2
bes_n_neigh = 3 
kll_final = []
y_change_final = []
for i in range(best_n_days,len(df_xmr)) : 
    g =[ df_xmr.iloc[j]["Close"] for j in range(i-best_n_days,i) ]
    kll_final.append(g)
for i in range(1,len(df_xmr)) : 
    y_change_final.append(df_xmr.iloc[i]["Close"] > df_xmr.iloc[i-1]["Close"])
knn_10 = KNeighborsClassifier(n_neighbors=bes_n_neigh)
knn_10.fit(kll_final,y_change_final[best_n_days-1:])
y_10_final = knn_10.predict(kll_final)

In [10]:
## f1 score for train data
ff_sc = f1_score(y_true=y_change_final[best_n_days-1:] , y_pred = y_10_final)
print(ff_sc)

0.8179669030732861


In [11]:
# download test data
df_xmr_test = yf.download(tickers = "XMR-USD",
                     start = "2023-09-04",
                     end = "2023-10-07" , 
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [12]:
## create input and output test data
kll_t = []
y_t = []
for i in range(best_n_days,len(df_xmr_test)) : 
    g = [df_xmr_test.iloc[j]["Close"] for j in range(i-best_n_days,i)]
    kll_t.append(g)
for i in range(1,len(df_xmr_test)) : 
    y_t.append(df_xmr_test.iloc[i]["Close"] > df_xmr_test.iloc[i-1]["Close"])

## predict output using trained model
y_pred_t = knn_10.predict(kll_t)

In [13]:
## scores for test data
ff_t = f1_score(y_true=y_t[best_n_days-1:] , y_pred = y_pred_t)
rr_t = recall_score(y_true=y_t[best_n_days-1:] , y_pred = y_pred_t)
pp_t = precision_score(y_true=y_t[best_n_days-1:] , y_pred = y_pred_t)
print("f1 score on test data :" , ff_t)
print("recall score on test data :" , rr_t)
print("precision score on test data :" , pp_t)

f1 score on test data : 0.717948717948718
recall score on test data : 0.7368421052631579
precision score on test data : 0.7
